<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/barrier_option_dupire_sensitivities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mathematical Description of the Financial Model

## Barrier Options under the Black-Scholes Model

A barrier option is a type of financial derivative whose payoff depends on whether the underlying asset's price reaches a specific barrier level during the option's life.


### Black-Scholes Formula for Barrier Options

The value of a barrier option can be derived by adjusting the standard Black-Scholes model with additional terms to account for the barrier conditions. The formulas for *up-and-out call* and *down-and-out put* options are given by:

$
C_{\text{up-out}} =
\begin{cases}
    0, & S \geq H \\\\
    S \Phi(d_1) - K e^{-rT} \Phi(d_2) - \left[ S \left(\frac{H}{S}\right)^{2\lambda} \Phi(x_1) - K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(x_2) \right], & S < H
\end{cases},
$

$
P_{\text{down-out}} =
\begin{cases}
    0, & S \leq H \\\\
    K e^{-rT} \Phi(-d_2) - S \Phi(-d_1) - \left[ K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(-x_2) - S \left(\frac{H}{S}\right)^{2\lambda} \Phi(-x_1) \right], & S > H
\end{cases},
$


### Parameter Definitions

$
d_1 = \frac{\ln(S / K) + (r + 0.5 \sigma^2)T}{\sigma \sqrt{T}}, \quad
d_2 = d_1 - \sigma \sqrt{T},
$

$
x_1 = \frac{\ln(S / H)}{\sigma \sqrt{T}} + \lambda \sigma \sqrt{T}, \quad
x_2 = x_1 - \sigma \sqrt{T},
$

$
\lambda = \frac{r + 0.5 \sigma^2}{\sigma^2}.
$

Here, $ S $ is the spot price, $ K $ is the strike price, $ H $ is the barrier level, $ r $ is the risk-free interest rate, $ T $ is the time to maturity, and $ \sigma $ is the local volatility. $ \Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution.


## Sensitivity Analysis using Automatic Differentiation

The sensitivity of the barrier option price to the local volatility surface $ \sigma_{\text{loc}}(K, T) $ is computed using automatic differentiation. The sensitivity is defined as:

$
\frac{\partial V}{\partial \sigma_{\text{loc}}(K, T)},
$

where $ V $ is the price of the barrier option. By leveraging automatic differentiation, this derivative is computed directly in the computational graph without requiring finite difference approximations.


In [ ]:
import torch
from torch.distributions.normal import Normal

# Barrier option pricing with the Black-Scholes formula
def barrier_option_price(S, K, T, r, sigma, H, option_type="call", barrier_type="up-and-out"):
    # S: Spot price (scalar or tensor)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma: Local volatility (tensor, same shape as K and T)
    # H: Barrier level (scalar)
    # option_type: "call" or "put"
    # barrier_type: "up-and-out" or "down-and-out"

    # Avoid division by zero for very small T
    T = torch.clamp(T, min=1e-6)

    # Standard normal distribution
    normal = Normal(0, 1)

    # Parameters for Black-Scholes formula
    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)

    # Parameters for the barrier option adjustment
    lambda_ = (r + 0.5 * sigma**2) / (sigma**2)
    x1 = torch.log(S / H) / (sigma * torch.sqrt(T)) + lambda_ * sigma * torch.sqrt(T)
    x2 = x1 - sigma * torch.sqrt(T)

    # Barrier option pricing logic
    if option_type == "call" and barrier_type == "up-and-out":
        # Up-and-out call
        return torch.where(S >= H, torch.zeros_like(K),  # Knocked out if spot >= barrier
                           S * normal.cdf(d1) - K * torch.exp(-r * T) * normal.cdf(d2) - \
                           (S * (H / S)**(2 * lambda_) * normal.cdf(x1) - K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(x2)))
    elif option_type == "put" and barrier_type == "down-and-out":
        # Down-and-out put
        return torch.where(S <= H, torch.zeros_like(K),  # Knocked out if spot <= barrier
                           K * torch.exp(-r * T) * normal.cdf(-d2) - S * normal.cdf(-d1) - \
                           (K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(-x2) - S * (H / S)**(2 * lambda_) * normal.cdf(-x1)))
    else:
        raise ValueError("Unsupported option or barrier type")

# Function to calculate sensitivities for barrier options
def calculate_barrier_sensitivities(S, K, T, r, sigma_surface, H, option_type="call", barrier_type="up-and-out", epsilon=1e-4):
    # S: Spot price (scalar)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma_surface: Local volatility surface (tensor of shape [num_strikes, num_times])
    # H: Barrier level (scalar)

    # Expand S, K, and T to match the shape of sigma_surface
    S_exp = S * torch.ones_like(sigma_surface)
    K_exp = K.unsqueeze(1).expand_as(sigma_surface)
    T_exp = T.unsqueeze(0).expand_as(sigma_surface)

    # Calculate the option price for the base volatility surface
    base_prices = barrier_option_price(S_exp, K_exp, T_exp, r, sigma_surface, H, option_type, barrier_type)

    # Sensitivities with respect to volatility surface
    sensitivity_sigma = torch.zeros_like(sigma_surface)

    # Loop over the volatility surface and calculate the derivatives (sensitivities)
    for i in range(sigma_surface.shape[0]):  # Loop over strikes
        for j in range(sigma_surface.shape[1]):  # Loop over times
            # Perturb the volatility surface slightly
            sigma_perturbed = sigma_surface.clone()
            sigma_perturbed[i, j] += epsilon

            # Calculate option price for the perturbed volatility surface
            perturbed_prices = barrier_option_price(S_exp, K_exp, T_exp, r, sigma_perturbed, H, option_type, barrier_type)

            # Compute sensitivity by finite difference
            sensitivity_sigma[i, j] = (perturbed_prices[i, j] - base_prices[i, j]) / epsilon

    return sensitivity_sigma

In [ ]:
# Example parameters
S = torch.tensor(100.0)  # Spot price
K = torch.linspace(90, 110, 10)  # Strike prices (from 90 to 110)
T = torch.linspace(0.1, 2.0, 5)  # Time to maturities (from 0.1 to 2 years)
r = torch.tensor(0.05)  # Risk-free rate (5%)
H = torch.tensor(120.0)  # Barrier level

# Example volatility surface (local volatility from Dupire model)
sigma_surface = torch.ones((10, 5)) * 0.2  # 10 strikes, 5 maturities

# Calculate sensitivities for barrier options
sensitivity_sigma = calculate_barrier_sensitivities(S, K, T, r, sigma_surface, H)

# Print the sensitivities
print("Sensitivity with respect to the volatility surface for barrier options:")
print(sensitivity_sigma)


In [ ]:
import torch
from torch.distributions.normal import Normal

# Barrier option pricing with the Black-Scholes formula
def barrier_option_price(S, K, T, r, sigma, H, option_type="call", barrier_type="up-and-out"):
    # S: Spot price (scalar or tensor)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma: Local volatility (tensor, same shape as K and T)
    # H: Barrier level (scalar)
    # option_type: "call" or "put"
    # barrier_type: "up-and-out" or "down-and-out"

    # Avoid division by zero for very small T
    T = torch.clamp(T, min=1e-6)

    # Standard normal distribution
    normal = Normal(0, 1)

    # Parameters for Black-Scholes formula
    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(T))
    d2 = d1 - sigma * torch.sqrt(T)

    # Parameters for the barrier option adjustment
    lambda_ = (r + 0.5 * sigma**2) / (sigma**2)
    x1 = torch.log(S / H) / (sigma * torch.sqrt(T)) + lambda_ * sigma * torch.sqrt(T)
    x2 = x1 - sigma * torch.sqrt(T)

    # Barrier option pricing logic
    if option_type == "call" and barrier_type == "up-and-out":
        return torch.where(S >= H, torch.zeros_like(K),  # Knocked out if spot >= barrier
                           S * normal.cdf(d1) - K * torch.exp(-r * T) * normal.cdf(d2) - \
                           (S * (H / S)**(2 * lambda_) * normal.cdf(x1) - K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(x2)))
    elif option_type == "put" and barrier_type == "down-and-out":
        return torch.where(S <= H, torch.zeros_like(K),  # Knocked out if spot <= barrier
                           K * torch.exp(-r * T) * normal.cdf(-d2) - S * normal.cdf(-d1) - \
                           (K * torch.exp(-r * T) * (H / S)**(2 * lambda_ - 2) * normal.cdf(-x2) - S * (H / S)**(2 * lambda_) * normal.cdf(-x1)))
    else:
        raise ValueError("Unsupported option or barrier type")

# Function to calculate sensitivities for barrier options using automatic differentiation
def calculate_barrier_sensitivities_autodiff(S, K, T, r, sigma_surface, H, option_type="call", barrier_type="up-and-out"):
    # S: Spot price (scalar)
    # K: Strike price (tensor)
    # T: Time to maturity (tensor)
    # r: Risk-free rate (scalar)
    # sigma_surface: Local volatility surface (tensor of shape [num_strikes, num_times])
    # H: Barrier level (scalar)

    # Expand S, K, and T to match the shape of sigma_surface
    S_exp = S * torch.ones_like(sigma_surface)
    K_exp = K.unsqueeze(1).expand_as(sigma_surface)
    T_exp = T.unsqueeze(0).expand_as(sigma_surface)

    # Enable gradient tracking for the sigma_surface
    sigma_surface = sigma_surface.clone().detach().requires_grad_(True)

    # Calculate the option price for the given volatility surface
    prices = barrier_option_price(S_exp, K_exp, T_exp, r, sigma_surface, H, option_type, barrier_type)

    # Perform automatic differentiation to compute sensitivities
    sensitivities = torch.autograd.grad(outputs=prices.sum(), inputs=sigma_surface, create_graph=True)[0]

    return sensitivities

In [ ]:
# Example parameters
S = torch.tensor(100.0)  # Spot price
K = torch.linspace(90, 110, 10)  # Strike prices (from 90 to 110)
T = torch.linspace(0.1, 2.0, 5)  # Time to maturities (from 0.1 to 2 years)
r = torch.tensor(0.05)  # Risk-free rate (5%)
H = torch.tensor(120.0)  # Barrier level

# Example volatility surface (local volatility from Dupire model)
sigma_surface = torch.ones((10, 5)) * 0.2  # 10 strikes, 5 maturities

# Calculate sensitivities for barrier options using automatic differentiation
sensitivity_sigma = calculate_barrier_sensitivities_autodiff(S, K, T, r, sigma_surface, H)

# Print the sensitivities
print("Sensitivity with respect to the volatility surface for barrier options:")
print(sensitivity_sigma)